<a href="https://colab.research.google.com/github/movieload2020/google-personal-colab/blob/main/Torrent_to_google_drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## <center><img src="https://i.ibb.co/4sSt9fM/py.jpg" alt="cover" border="0"> </center>

# *STEP 1 : গুগল ড্রাইভ মাউন্ট করুন*

In [ ]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Mount Gdrive to /content/drive</h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive! 
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

# *STEP 2 : পাইথন লাইব্রেরি ইন্সটল করুন*

In [ ]:
#@title
#@markdown <h3>⬅️ Click Here to START server</h3>
 
 
!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent
!apt install python3-libtorrent
 
import libtorrent as lt
 
ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []
 
from IPython.display import HTML, clear_output
 
clear_output()
 
print("Server Started Successfully")

# *STEP 3 : ম্যাগনেট লিংক থেকে টরেন্ট ফাইল ডাউনলোড (My Drive এর জন্য)*
 
# ***Following steps for Google Drive Upload (Personal Drive)***
<br><center><img src='https://i.ibb.co/2ZHRQqH/Personal-drive.png' height="100" /></center>

In [ ]:
#@title
#@markdown <h3>⬅️ Click Here to </h3>
#@markdown <h3>Add Magnet Link</h3>
#@markdown <h5>You can run this cell to add more files as many times as you want.</h5>
#@markdown <h5>You can download parallel file Using Magnet Link.</h5><br>
 
 
params = {"save_path": "/content/drive/My Drive/Torrent"}
 
while True:
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )

# *STEP 3 : ম্যাগনেট লিংক থেকে টরেন্ট ফাইল ডাউনলোড (Shared Drive এর জন্য)*

# ***Following steps for Team Drive Upload (Shared Drive)***
<br><center><img src='https://i.ibb.co/9GZ01LL/Shared-Drive.png' height="100" /></center> 



In [ ]:
#@title
#@markdown <h3>⬅️ Click Here to </h3>
#@markdown <h3>Add Magnet Link</h3>
#@markdown <h5>You can run this cell to add more files as many times as you want.</h5>
#@markdown <h5>You can download parallel file Using Magnet Link.</h5><br>

#@markdown <h3>Enter the path of Share Drive(where you want to download):</h3>
path = "" #@param {type:"string"}
check=[]
params = {"save_path": path}

while True:
    magnet_link = input("Enter Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break
    downloads.append(
        lt.add_magnet_uri(ses, magnet_link, params)
    )

clear_output()

print("Started Successfully")   


# *STEP 4 : ক্ল্যাউড ট্রান্সফার শুরু করুন*

In [ ]:
#@title
#@markdown <h3>⬅️ Click Here to Start Download</h3>
import time
from IPython.display import display
import ipywidgets as widgets
 
state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]
 
layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)
 
while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()
 
            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)